In [1]:
import os
os.chdir('..')

In [15]:
import pandas as pd

In [6]:
from src.common.experiment_runners import run_experiments_efga
from src.common.fitness import schwefel, griewank, rastrigin, ackley
from src.common.scales import scales

In [3]:
%%time
run_experiments_efga(n_experiments=5, use_approx=False)

100%|██████████| 5/5 [01:31<00:00, 18.28s/it]

CPU times: user 11min 27s, sys: 2.34 s, total: 11min 30s
Wall time: 1min 31s


(     best_fitness  avg_fitness  epoch  ncalls  seed
 0      991.996848  2022.040599     32   33282     0
 1     1182.485222  1987.007344     31   33282     0
 2     1375.962803  1970.678342     33   33282     0
 3     1059.889019  1965.975528     33   33282     0
 4     1060.381897  1951.329234     38   33282     0
 ..            ...          ...    ...     ...   ...
 495      7.404140  1196.119007     23   32488     4
 496      7.271415  1196.114243     30   32488     4
 497      7.273023  1195.881770     31   32488     4
 498      8.452040  1195.880789     22   32488     4
 499      7.271415  1195.868651     28   32488     4
 
 [2500 rows x 5 columns],
 {'n_experiments': 5,
  'fitness_fn': 'schwefel',
  'population_scale': 500,
  'mutation_scale': 0.5,
  'N': 500,
  'epochs': 500,
  'n_terms_params': 3,
  'n_terms_priority': 3,
  'use_approx': False})

In [4]:
%%time
run_experiments_efga(n_experiments=5, use_approx=True)

100%|██████████| 5/5 [00:24<00:00,  4.95s/it]

CPU times: user 3min 57s, sys: 729 ms, total: 3min 57s
Wall time: 24.7 s


(     best_fitness  avg_fitness  epoch  ncalls  seed
 0      991.996848  2022.040599     32   32978     0
 1     1182.485222  1987.007344     31   32978     0
 2     1375.962803  1970.678342     33   32978     0
 3     1059.889019  1965.975528     33   32978     0
 4     1060.381897  1951.329234     38   32978     0
 ..            ...          ...    ...     ...   ...
 495      7.561482  1224.197867     37   32458     4
 496      7.500575  1224.195131     26   32458     4
 497     13.269308  1224.191517     27   32458     4
 498      7.561482  1224.183431     25   32458     4
 499      7.382368  1224.183985     24   32458     4
 
 [2500 rows x 5 columns],
 {'n_experiments': 5,
  'fitness_fn': 'schwefel',
  'population_scale': 500,
  'mutation_scale': 0.5,
  'N': 500,
  'epochs': 500,
  'n_terms_params': 3,
  'n_terms_priority': 3,
  'use_approx': True})

In [158]:
history_general = []
for do_approx in [True, False]:
    for fitness_fn in [schwefel, rastrigin, ackley, griewank]:
        fn_name = fitness_fn.__name__
        history_general.append(
            run_experiments_efga(n_experiments=15, fitness_fn=fitness_fn, population_scale=scales[fn_name][0], 
                                 mutation_scale=scales[fn_name][1], N=1000, use_approx=do_approx)
        )

100%|██████████| 15/15 [04:53<00:00, 19.56s/it]


In [159]:
import pickle as pkl

with open('backup_approx_non_approx', 'wb') as f:
    pkl.dump(history_general, f)

In [160]:
from copy import deepcopy


rcrds_df = []
for i in history_general:
    log, params = i
    records = log.groupby('seed').min()['best_fitness'].tolist()

    for j in range(len(records)):
        score = records[j]
        records[j] = deepcopy(params)
        records[j]['score']  = score
        records[j]['seed'] = j

    rcrds_df += records
rcrds_df =  pd.DataFrame(rcrds_df)

In [161]:
rcrds_df.shape

(120, 11)

In [162]:
from scipy import stats

In [163]:
stats.wilcoxon(rcrds_df[rcrds_df.use_approx].score, rcrds_df[~rcrds_df.use_approx].score)

WilcoxonResult(statistic=913.0, pvalue=0.9882530056508173)

In [164]:
rcrds_df[rcrds_df.use_approx].shape

(60, 11)

In [165]:
rcrds_df[~rcrds_df.use_approx].shape

(60, 11)

In [166]:
rcrds_df[rcrds_df.use_approx].score.shape

(60,)

In [167]:
rcrds_df[~rcrds_df.use_approx].score.shape

(60,)

In [168]:
rcrds_df

,n_experiments,fitness_fn,population_scale,mutation_scale,N,epochs,n_terms_params,n_terms_priority,use_approx,score,seed
0,15,schwefel,500.0,0.5,1000,500,3,3,True,103.889159,0
1,15,schwefel,500.0,0.5,1000,500,3,3,True,58.913897,1
2,15,schwefel,500.0,0.5,1000,500,3,3,True,4.380583,2
3,15,schwefel,500.0,0.5,1000,500,3,3,True,20.053225,3
4,15,schwefel,500.0,0.5,1000,500,3,3,True,7.267605,4
...,...,...,...,...,...,...,...,...,...,...,...
115,15,griewank,600.0,0.2,1000,500,3,3,False,0.340346,10
116,15,griewank,600.0,0.2,1000,500,3,3,False,0.172359,11
117,15,griewank,600.0,0.2,1000,500,3,3,False,0.134583,12
118,15,griewank,600.0,0.2,1000,500,3,3,False,0.458418,13


In [169]:
rcrds_df.groupby(['use_approx', 'fitness_fn']).median()['score']

use_approx  fitness_fn
False       ackley        19.367676
            griewank       0.230240
            rastrigin      1.155170
            schwefel       7.796173
True        ackley        19.573533
            griewank       0.386163
            rastrigin      1.422504
            schwefel      20.053225
Name: score, dtype: float64

In [170]:
rcrds_df[rcrds_df.use_approx].score.to_numpy() == rcrds_df[~rcrds_df.use_approx].score.to_numpy()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [171]:
from scikit_posthocs import posthoc_nemenyi_friedman
import numpy as np

In [172]:
a = np.vstack([rcrds_df[rcrds_df.use_approx].score, rcrds_df[~rcrds_df.use_approx].score,]).T

In [173]:
posthoc_nemenyi_friedman(a)

,0,1
0,1.0,0.9
1,0.9,1.0


In [91]:
! pip install pingouin


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.5/979.5 kB 4.6 MB/s eta 0:00:0000:0100:01
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 7.7 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 8.1 MB/s eta 0:00:00
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=cb40f3b92a4b79eb628b8fabee768ebf6f264b569fffc027c9d332cac4a3d10f
  Stored in directory: /home/oleksandr/.var/app/com.visualstudio.code/cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [174]:
pivoted = pd.pivot(rcrds_df, values=['score'], columns='use_approx', index=['seed', 'fitness_fn']).reset_index().groupby(['seed', 'fitness_fn']).median()
pivoted

score            
use_approx            False        True
seed fitness_fn                        
0    ackley       20.000000   20.002020
     griewank      0.396448    0.648642
     rastrigin     1.609714    0.191978
     schwefel    103.959120  103.889159
1    ackley       18.611313   17.332977
     griewank      0.498947    0.437041
     rastrigin     0.131326    0.003290
     schwefel     58.890322   58.913897
2    ackley       18.841638   19.323001
     griewank      0.230240    0.679735
     rastrigin     1.580411    0.401520
     schwefel      4.393365    4.380583
3    ackley       20.001432   20.007265
     griewank      0.462685    1.325130
     rastrigin     1.586256    1.422504
     schwefel     20.067424   20.053225
4    ackley       19.367676   20.000000
     griewank      0.812470    0.927718
     rastrigin     0.234544    0.269386
     schwefel      7.268558    7.267605
5    ackley       17.667019   19.573533
     griewank      0.163263    0.199495
     rastrigin     3.214526    0.990350
     schwefel      6.515825    6.487151
6    ackley       20.008587   20.010735
     griewank      0.202726    0.284701
     rastrigin     0.193716    2.366329
     schwefel      1.785448   70.177829
7    ackley       18.573870   16.737764
     griewank      0.454626    0.482367
     rastrigin     0.305368    0.191500
     schwefel     29.017042   28.842070
8    ackley       20.000000   20.000324
     griewank      0.137988    0.123792
     rastrigin     0.937329    1.562327
     schwefel     50.845280   50.815845
9    ackley       19.298998   20.021357
     griewank      0.143403    0.136104
     rastrigin     0.640486    0.185980
     schwefel     27.387348   27.364151
10   ackley       20.000000   13.587972
     griewank      0.340346    0.386163
     rastrigin     4.567425    1.655757
     schwefel      2.173943    2.173670
11   ackley       20.000000   15.608677
     griewank      0.172359    0.094762
     rastrigin     0.995717    3.110562
     schwefel      1.073765   11.662441
12   ackley       10.760695   20.024439
     griewank      0.134583    0.367551
     rastrigin     1.155170    1.433690
     schwefel      7.796173    7.837019
13   ackley       18.406288   17.655839
     griewank      0.458418    0.421007
     rastrigin     2.889829    1.824509
     schwefel      7.510807    1.946537
14   ackley       20.000000   19.538653
     griewank      0.150942    0.308474
     rastrigin     1.271594    1.582899
     schwefel     83.867780   83.868693

In [175]:
pivoted.columns

MultiIndex([('score', False),
            ('score',  True)],
           names=[None, 'use_approx'])

In [176]:
pgdf = pd.DataFrame({'scores_'})

In [177]:
rcrds_df

,n_experiments,fitness_fn,population_scale,mutation_scale,N,epochs,n_terms_params,n_terms_priority,use_approx,score,seed
0,15,schwefel,500.0,0.5,1000,500,3,3,True,103.889159,0
1,15,schwefel,500.0,0.5,1000,500,3,3,True,58.913897,1
2,15,schwefel,500.0,0.5,1000,500,3,3,True,4.380583,2
3,15,schwefel,500.0,0.5,1000,500,3,3,True,20.053225,3
4,15,schwefel,500.0,0.5,1000,500,3,3,True,7.267605,4
...,...,...,...,...,...,...,...,...,...,...,...
115,15,griewank,600.0,0.2,1000,500,3,3,False,0.340346,10
116,15,griewank,600.0,0.2,1000,500,3,3,False,0.172359,11
117,15,griewank,600.0,0.2,1000,500,3,3,False,0.134583,12
118,15,griewank,600.0,0.2,1000,500,3,3,False,0.458418,13


In [178]:
rcrds_df['id'] = rcrds_df.seed.astype(str) + rcrds_df.fitness_fn

In [185]:
import pingouin as pg

pg.wilcoxon(pivoted.iloc[:, 0], pivoted.iloc[:, 1])

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,913.0,two-sided,0.99119,0.002186,0.484583
